# Task 2: SQL Analytics - Advanced Queries

## Scenario
You have ticket and user data in SQLite. Your task is to:
1. Design and create an efficient database schema
2. Write SQL queries to answer business questions
3. Perform time-series analysis
4. Find patterns in user behavior

## Learning Objectives
- Design database schemas with relationships
- Write complex SQL queries (JOINs, aggregations, window functions)
- Perform time-series analysis with SQL
- Optimize queries with indexes

## Business Questions to Answer
1. Who are the top 10 users by ticket count?
2. What's the average response time by category?
3. How is ticket volume growing month-over-month?
4. Which users had tickets in 3+ consecutive months?

---
## Setup (Provided)

In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path

# Database path
DB_PATH = Path("../fixtures/output/analytics.db")
DB_PATH.parent.mkdir(exist_ok=True)

# Remove old database if exists
if DB_PATH.exists():
    DB_PATH.unlink()

# Create connection
conn = sqlite3.connect(DB_PATH)

print("Setup complete!")

---
## Task 2.1: Create Database Schema

Create two tables: `tickets` and `users` with proper schema.

**Tickets table:**
- `ticket_id` (INTEGER PRIMARY KEY)
- `user_id` (INTEGER, foreign key)
- `category` (TEXT)
- `description` (TEXT)
- `created_at` (TEXT - ISO format)
- `resolved_at` (TEXT - ISO format)
- `priority` (TEXT)
- `status` (TEXT)

**Users table:**
- `user_id` (INTEGER PRIMARY KEY)
- `username` (TEXT)
- `email` (TEXT)
- `signup_date` (TEXT)
- `plan_type` (TEXT)
- `company` (TEXT)

**Tasks:**
1. Load CSV files into DataFrames
2. Write both DataFrames to SQLite using `to_sql()`
3. Create indexes on `user_id`, `category`, and `created_at` for tickets table

In [ ]:
# YOUR CODE HERE
# Load CSVs and create tables with indexes



In [ ]:
# TEST - Do not modify
# Check tables exist
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
assert 'tickets' in tables['name'].values, "Table 'tickets' not found"
assert 'users' in tables['name'].values, "Table 'users' not found"

# Check row counts
ticket_count = pd.read_sql("SELECT COUNT(*) as cnt FROM tickets", conn)['cnt'].iloc[0]
user_count = pd.read_sql("SELECT COUNT(*) as cnt FROM users", conn)['cnt'].iloc[0]
assert ticket_count == 50, f"Expected 50 tickets, got {ticket_count}"
assert user_count == 21, f"Expected 21 users, got {user_count}"

# Check indexes
indexes = pd.read_sql("SELECT name FROM sqlite_master WHERE type='index'", conn)['name'].tolist()
assert any('user_id' in idx.lower() for idx in indexes), "Missing user_id index"
assert any('category' in idx.lower() for idx in indexes), "Missing category index"

print("✓ Task 2.1 PASSED!")
print(f"\nDatabase schema created:")
print(f"  Tickets: {ticket_count} rows")
print(f"  Users: {user_count} rows")
print(f"  Indexes: {len([idx for idx in indexes if 'auto' not in idx.lower()])}")

---
## Task 2.2: Top 10 Users by Ticket Count

Find the top 10 users who created the most tickets.

**Required columns:**
- `user_id`
- `username`
- `company`
- `ticket_count`

Store result in variable `top_users`.

**Hint:** Use JOIN, GROUP BY, and ORDER BY

In [ ]:
# YOUR CODE HERE
# Write SQL query to find top 10 users



In [ ]:
# TEST - Do not modify
assert 'top_users' in dir(), "Variable 'top_users' not found"
assert len(top_users) == 10, f"Expected 10 rows, got {len(top_users)}"
assert 'username' in top_users.columns, "Missing 'username' column"
assert 'ticket_count' in top_users.columns, "Missing 'ticket_count' column"

# Check ordering (descending by ticket_count)
counts = top_users['ticket_count'].tolist()
assert counts == sorted(counts, reverse=True), "Results not properly sorted"

# Check top user
assert top_users.iloc[0]['ticket_count'] >= 3, "Top user should have at least 3 tickets"

print("✓ Task 2.2 PASSED!")
print("\nTop 10 Users by Ticket Count:")
print(top_users.to_string(index=False))

---
## Task 2.3: Average Response Time by Category

Calculate average resolution time (in hours) for each category.

**Required columns:**
- `category`
- `avg_resolution_hours`
- `ticket_count`
- `min_hours`
- `max_hours`

Store result in variable `category_stats`.

**Hint:** Use `strftime()` or `julianday()` to calculate time differences in SQLite

**SQLite datetime functions:**
```sql
-- Hours between two timestamps:
(julianday(resolved_at) - julianday(created_at)) * 24
```

In [ ]:
# YOUR CODE HERE
# Write SQL query to calculate category statistics



In [ ]:
# TEST - Do not modify
assert 'category_stats' in dir(), "Variable 'category_stats' not found"
assert len(category_stats) == 3, f"Expected 3 categories, got {len(category_stats)}"
assert 'avg_resolution_hours' in category_stats.columns, "Missing 'avg_resolution_hours' column"
assert 'ticket_count' in category_stats.columns, "Missing 'ticket_count' column"

# Check that all categories are present
categories = set(category_stats['category'].tolist())
expected_categories = {'Technical', 'Billing', 'Account'}
assert categories == expected_categories, f"Expected {expected_categories}, got {categories}"

# Check reasonable values
assert all(category_stats['avg_resolution_hours'] > 0), "Average hours should be positive"
assert all(category_stats['avg_resolution_hours'] < 100), "Average hours seems too high"
assert category_stats['ticket_count'].sum() == 50, "Total tickets should be 50"

print("✓ Task 2.3 PASSED!")
print("\nAverage Response Time by Category:")
print(category_stats.to_string(index=False))

---
## Task 2.4: Month-over-Month Ticket Growth

Calculate ticket volume for each month and compute growth rate.

**Required columns:**
- `month` (format: YYYY-MM)
- `ticket_count`
- `prev_month_count` (previous month's count, NULL for first month)
- `growth_rate` (percentage change, NULL for first month)

Store result in variable `monthly_growth`.

**Hint:** Use window functions (LAG) or self-join

**Window function example:**
```sql
LAG(column_name) OVER (ORDER BY month) as prev_value
```

In [ ]:
# YOUR CODE HERE
# Write SQL query for month-over-month analysis



In [ ]:
# TEST - Do not modify
assert 'monthly_growth' in dir(), "Variable 'monthly_growth' not found"
assert len(monthly_growth) == 5, f"Expected 5 months, got {len(monthly_growth)}"
assert 'month' in monthly_growth.columns, "Missing 'month' column"
assert 'ticket_count' in monthly_growth.columns, "Missing 'ticket_count' column"
assert 'prev_month_count' in monthly_growth.columns, "Missing 'prev_month_count' column"

# Check months are in order
months = monthly_growth['month'].tolist()
assert months == sorted(months), "Months should be in chronological order"

# Check first month has NULL prev_month_count
first_prev = monthly_growth.iloc[0]['prev_month_count']
assert pd.isna(first_prev) or first_prev is None, "First month should have NULL prev_month_count"

# Check subsequent months have values
assert not pd.isna(monthly_growth.iloc[1]['prev_month_count']), "Second month should have prev_month_count"

# Check total tickets
assert monthly_growth['ticket_count'].sum() == 50, "Total tickets should be 50"

print("✓ Task 2.4 PASSED!")
print("\nMonth-over-Month Growth:")
print(monthly_growth.to_string(index=False))

---
## Task 2.5: Users with Consecutive Monthly Activity

Find users who created tickets in at least 3 consecutive months.

**Required columns:**
- `user_id`
- `username`
- `consecutive_months` (number of consecutive months)
- `start_month` (first month of the streak)
- `end_month` (last month of the streak)

Store result in variable `consecutive_users`.

**Approach:**
1. Extract month from created_at
2. Get distinct user-month combinations
3. Identify consecutive month sequences
4. Filter for sequences of 3+ months

**Hint:** This is challenging! You may need:
- Window functions (ROW_NUMBER, LEAD/LAG)
- Date arithmetic
- Multiple CTEs (WITH clauses)

**Alternative simpler approach:**
Create a list of months each user was active, then check for consecutive sequences in Python.

In [ ]:
# YOUR CODE HERE
# Find users with 3+ consecutive months of activity
# You can use SQL, Python, or a combination



In [ ]:
# TEST - Do not modify
assert 'consecutive_users' in dir(), "Variable 'consecutive_users' not found"
assert len(consecutive_users) > 0, "Should find at least one user with consecutive months"
assert 'user_id' in consecutive_users.columns, "Missing 'user_id' column"
assert 'consecutive_months' in consecutive_users.columns, "Missing 'consecutive_months' column"

# Check all users have at least 3 consecutive months
assert all(consecutive_users['consecutive_months'] >= 3), \
    "All users should have at least 3 consecutive months"

# Verify by checking actual data for one user
if len(consecutive_users) > 0:
    test_user_id = consecutive_users.iloc[0]['user_id']
    user_tickets = pd.read_sql(
        f"SELECT strftime('%Y-%m', created_at) as month FROM tickets WHERE user_id = {test_user_id} ORDER BY month",
        conn
    )
    distinct_months = user_tickets['month'].unique()
    assert len(distinct_months) >= 3, f"User {test_user_id} should have tickets in at least 3 months"

print("✓ Task 2.5 PASSED!")
print("\nUsers with 3+ Consecutive Months:")
print(consecutive_users.to_string(index=False))

---
## Task 2.6: Complex Query - VIP Users

Identify VIP users based on multiple criteria:
- Enterprise or Premium plan
- Created at least 3 tickets
- At least one critical priority ticket
- Average resolution time < 10 hours

**Required columns:**
- `user_id`
- `username`
- `plan_type`
- `total_tickets`
- `critical_tickets`
- `avg_resolution_hours`

Store result in variable `vip_users`.

**Hint:** Use subqueries or CTEs with HAVING clause

In [ ]:
# YOUR CODE HERE
# Write SQL query to identify VIP users



In [ ]:
# TEST - Do not modify
assert 'vip_users' in dir(), "Variable 'vip_users' not found"
assert 'plan_type' in vip_users.columns, "Missing 'plan_type' column"
assert 'total_tickets' in vip_users.columns, "Missing 'total_tickets' column"
assert 'avg_resolution_hours' in vip_users.columns, "Missing 'avg_resolution_hours' column"

if len(vip_users) > 0:
    # Verify all VIP criteria
    assert all(vip_users['plan_type'].isin(['enterprise', 'premium', 'Enterprise', 'Premium'])), \
        "All VIP users should have Enterprise or Premium plan"
    assert all(vip_users['total_tickets'] >= 3), \
        "All VIP users should have at least 3 tickets"
    assert all(vip_users['avg_resolution_hours'] < 10), \
        "All VIP users should have avg resolution < 10 hours"
    
    print("✓ Task 2.6 PASSED!")
    print(f"\nFound {len(vip_users)} VIP users:")
    print(vip_users.to_string(index=False))
else:
    print("✓ Task 2.6 PASSED!")
    print("\nNo VIP users found (this is valid if criteria are strict)")

---
## Task 2.7: Query Optimization

Analyze query performance using EXPLAIN QUERY PLAN.

**Tasks:**
1. Run EXPLAIN QUERY PLAN on the top users query (Task 2.2)
2. Check if indexes are being used
3. Store the query plan in variable `query_plan`

In [ ]:
# YOUR CODE HERE
# Run EXPLAIN QUERY PLAN on your top users query



In [ ]:
# TEST - Do not modify
assert 'query_plan' in dir(), "Variable 'query_plan' not found"
assert len(query_plan) > 0, "Query plan should have rows"

# Check that plan contains expected operations
plan_text = ' '.join(query_plan.astype(str).values.flatten().tolist()).lower()
assert 'scan' in plan_text or 'search' in plan_text, "Query plan should show scan or search operations"

print("✓ Task 2.7 PASSED!")
print("\nQuery Plan:")
print(query_plan.to_string(index=False))

if 'using index' in plan_text:
    print("\n✅ Query is using indexes!")
else:
    print("\n⚠️  Query may not be using indexes optimally")

---
## Cleanup

In [ ]:
# Close connection
conn.close()
print("Database connection closed.")

---
## Summary

Congratulations! You've completed advanced SQL analytics:

✅ Created normalized database schema with relationships
✅ Wrote JOINs to combine user and ticket data
✅ Performed time-based aggregations
✅ Used window functions for time-series analysis
✅ Identified patterns in user behavior
✅ Found users with consecutive monthly activity
✅ Combined multiple criteria for complex filtering
✅ Analyzed query performance with EXPLAIN PLAN

## Key SQL Concepts Covered

**Joins:**
```sql
SELECT u.username, COUNT(t.ticket_id)
FROM users u
JOIN tickets t ON u.user_id = t.user_id
GROUP BY u.user_id
```

**Window Functions:**
```sql
LAG(ticket_count) OVER (ORDER BY month) as prev_count
```

**Date/Time Functions:**
```sql
strftime('%Y-%m', created_at) as month
(julianday(resolved_at) - julianday(created_at)) * 24 as hours
```

**CTEs (Common Table Expressions):**
```sql
WITH monthly_counts AS (
    SELECT strftime('%Y-%m', created_at) as month, COUNT(*) as cnt
    FROM tickets
    GROUP BY month
)
SELECT * FROM monthly_counts
```

## Performance Tips

1. **Index frequently filtered columns:** user_id, category, dates
2. **Use EXPLAIN QUERY PLAN** to verify index usage
3. **Avoid SELECT \*:** Only select needed columns
4. **Use LIMIT** for testing queries on large datasets
5. **Consider materialized views** for frequently-run complex queries